In [9]:
import torch
import pandas as pd
from tqdm import tqdm
from glob import glob
from utils.util import *
from utils.dataset import MaskTestDataset
from model_utils.model import load_model
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np

In [16]:
args = {
        "MODEL": 'efficientnet-b3', 
        "SAVED_DIR": "/opt/ml/code/seunghyun/baseline/checkpoint/2022-02-26_19:56:25", # 추론하고자하는 모델이 저장된 상위 폴더 경로
        "RESIZE": [312, 312], 
        "BATCH_SIZE": 64,
        "RANDOM_SEED": 42,
        "DEVICE": "cuda",
        "CRITERION": 'focal',
        "RANDOM_SEED": 42,
        "NUM_CLASSES": 18,
}

In [17]:
# 모델 경로 로드
save_paths = sorted(glob(args["SAVED_DIR"] + "/*"))
save_paths

['/opt/ml/code/seunghyun/baseline/checkpoint/2022-02-26_19:56:25/Fold1_Epoch8_0.827_cross_entropy.tar',
 '/opt/ml/code/seunghyun/baseline/checkpoint/2022-02-26_19:56:25/Fold2_Epoch20_0.833_cross_entropy.tar',
 '/opt/ml/code/seunghyun/baseline/checkpoint/2022-02-26_19:56:25/Fold3_Epoch7_0.806_cross_entropy.tar',
 '/opt/ml/code/seunghyun/baseline/checkpoint/2022-02-26_19:56:25/Fold4_Epoch12_0.782_cross_entropy.tar',
 '/opt/ml/code/seunghyun/baseline/checkpoint/2022-02-26_19:56:25/Fold5_Epoch9_0.804_cross_entropy.tar']

In [6]:

# submission file 로드
info = pd.read_csv("/opt/ml/input/data/eval/info.csv")


In [7]:
for idx, path in enumerate(save_paths):
    model = load_model(args)
    model.to(args["DEVICE"])

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()
    print(f"Path: {path}")
    fold_num = path.split("/")[-1].split("_")[0]
    test_dataset = MaskTestDataset(info, get_valid_transform(args))
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2, drop_last=False)

    preds = []
    with torch.no_grad():
        for idx, (images, id_) in enumerate(tqdm(test_loader, total=len(test_loader))):
            images = images.to("cuda")
            pred = model(images)
            pred = F.softmax(pred, dim=-1)
            preds.extend(pred.detach().cpu().numpy())
    info[fold_num] = preds

print(f'Inference Done!')
info['ans'] = (info['Fold1'] + info['Fold2'] + info['Fold3'] + info['Fold4'] + info['Fold5'])/5
sub_info = info.copy()
sub_info = sub_info.drop(['Fold1', 'Fold2', 'Fold3', 'Fold4', 'Fold5'], axis=1)
sub_info['ans'] = [np.argmax(i) for i in sub_info['ans']]
sub_info.to_csv("submission/fold_sub.csv", index=False)

Path: /opt/ml/code/seunghyun/baseline/checkpoint/2022-02-26_19:56:25/Fold3_Epoch7_0.806_cross_entropy.tar


100%|██████████| 197/197 [00:40<00:00,  4.92it/s]


Path: /opt/ml/code/seunghyun/baseline/checkpoint/2022-02-26_19:56:25/Fold2_Epoch20_0.833_cross_entropy.tar


100%|██████████| 197/197 [00:40<00:00,  4.85it/s]


Path: /opt/ml/code/seunghyun/baseline/checkpoint/2022-02-26_19:56:25/Fold5_Epoch9_0.804_cross_entropy.tar


...

Path: /opt/ml/code/seunghyun/baseline/checkpoint/2022-02-26_19:56:25/Fold4_Epoch12_0.782_cross_entropy.tar


100%|██████████| 197/197 [00:40<00:00,  4.85it/s]


Path: /opt/ml/code/seunghyun/baseline/checkpoint/2022-02-26_19:56:25/Fold1_Epoch8_0.827_cross_entropy.tar


100%|██████████| 197/197 [00:40<00:00,  4.91it/s]

Inference Done!
